In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
DT=1
optimize_model=False
Make_submission=True
n_estimators=150 #400 #500  #1500
max_depth=2 #4 #12  #8

In [ ]:
train=pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/train.csv")
train.head()

For some countries we also have data for individual regions

In [ ]:
train[train['Province_State'].notna()].groupby(['Country_Region'], sort=False)['Province_State'].nunique()

Grouping country names and provinces into variable - location

In [ ]:
def add_location(df_old):
    df=df_old.copy()
    df['Date']=pd.to_datetime(df['Date'])
    df['Country_Region']=df['Country_Region'].fillna('')
    df['Province_State']=df['Province_State'].fillna('')
    df['location']=df['Province_State'].astype('str')+" "+df['Country_Region'].astype('str')
    return df

In [ ]:
train=add_location(train)

Locations with less than 5 cases:

In [ ]:
max_cases_old=train[train['Date']<'2020-03-22'].groupby(['location'], sort=False)['ConfirmedCases'].max()
max_cases=train.groupby(['location'], sort=False)['ConfirmedCases'].max()

In [ ]:
train.set_index('location',inplace=True)

train['day_of_year']=train['Date'].dt.dayofyear
train['day_of_week']=train['Date'].dt.dayofweek



first_day=train[(train['ConfirmedCases']>0)].groupby(['location'], sort=False)['day_of_year'].min()
first_day.rename('first_day',inplace=True)

In [ ]:
def add_days_passed(df_old,first_day):
    df=df_old.copy()
    df=pd.concat([df,first_day],axis=1,join='inner')
    df['days_passed']=df['day_of_year']-df['first_day']
    df.drop(columns=['first_day'],inplace=True)
    df['location']=df.index
    df.set_index('Id',inplace=True)
    df['Id']=df.index
    return df

 


In [ ]:
train=add_days_passed(train,first_day)

train.head()

In [ ]:
country_stat=pd.read_csv('../input/countryinfo/covid19countryinfo.csv')
country_stat = country_stat[country_stat['region'].isnull()] 

def add_country_stat(old_df,country_stat):
    df=old_df.copy()
    df=df.merge(country_stat[['country','pop','medianage','sex65plus','lung','smokers','density']],left_on=['Country_Region'],right_on=['country'],how='left')
    df.drop(columns=['country'],inplace=True)
    
    df['pop']=df['pop'].fillna(1000)
    df['pop']=df['pop'].apply(lambda x: int(str(x).replace(',', '')))
    #df['gdp2019']=df['gdp2019'].fillna(0)
    #df['gdp2019']=df['gdp2019'].apply(lambda x: int(str(x).replace(',', '')))
    #df['gdp2019']=df['gdp2019']/df['pop']
    
    
    df['density']=df['density'].fillna(0)
    df['medianage']=df['medianage'].fillna(0)
    #df['sexratio']=df['sexratio'].fillna(1)
    df['sex65plus']=df['sex65plus'].fillna(1)
    df['lung']=df['lung'].fillna(24)
    df['smokers']=df['smokers'].fillna(24)
    #df['lung']=df['lung']*df['pop']
    
    return df
    

train=add_country_stat(train,country_stat)

In [ ]:
country_stat.info()

In [ ]:
border_info=pd.read_csv("https://raw.githubusercontent.com/geodatasource/country-borders/master/GEODATASOURCE-COUNTRY-BORDERS.CSV")
border_info.drop(columns=["country_code","country_border_code"],inplace=True)
border_info.replace({'United States of America':'US',
                    'United Kingdom of Great Britain and Northern Ireland':'United Kingdom',
                    'Bolivia (Plurinational State Of)':'Bolivia',
                    'Brunei Darussalam':'Brunei',
                    'Gambia (the)':'Gambia',
                     'Congo':'Congo (Kinshasa)',
                     'Cote d’Ivoire':"Cote d'Ivoire",
                     "Iran (Islamic Republic of)":'Iran',
                     "Korea (the Republic of)":'Korea, South',
                    "Lao People's Democratic Republic":'Laos',
                     "Moldova (the Republic of)":'Moldova',
                     "Russian Federation":'Russia',
                    "Syrian Arab Republic":'Syria',
                     "Taiwan (Province of China)":'Taiwan*',
                    "Tanzania (the United Republic of)":'Tanzania',
                     "Venezuela (Bolivarian Republic of)":'Venezuela',
                     "Viet Nam":'Vietnam'},inplace=True)
border_info=border_info.fillna("")
border_info.to_csv("border_info.csv")


In [ ]:
from itertools import product as it_product
def expand_grid(data_dict):
  rows = it_product(*data_dict.values())
  return pd.DataFrame.from_records(rows, columns=data_dict.keys())

In [ ]:
skel=expand_grid({'Index':border_info.index,'Date':train['Date'].unique()})
skel.info()

In [ ]:
country_info=train.groupby(['Date','Country_Region'])['ConfirmedCases'].sum()

In [ ]:
skel=expand_grid({'Index':border_info.index,'Date':train['Date'].unique()})

skel=skel.merge(border_info, how='inner', left_on=['Index'],right_index=True)
skel=skel.merge(country_info, how='inner', 
                left_on=['Date','country_border_name'],right_on=['Date','Country_Region'])

In [ ]:
from datetime import timedelta
skel['Date']=skel['Date']+timedelta(days=DT)
border_cases=skel.groupby(['country_name','Date'])['ConfirmedCases'].sum()
len(skel['country_name'].unique())

In [ ]:
train=train.merge(border_cases, how='left', left_on=['Country_Region','Date'],right_on=['country_name','Date'])
train['ConfirmedCases_y']=train['ConfirmedCases_y'].fillna(0)
train.rename(columns={'ConfirmedCases_y':'ConfirmedCases_neighbors','ConfirmedCases_x':'ConfirmedCases'},inplace=True)

In [ ]:
big_train = pd.concat([train,pd.get_dummies(train['location'], prefix='loc')],axis=1)
big_train['ConfirmedCases_neighbors']=np.log1p(big_train['ConfirmedCases_neighbors'])
big_train.reset_index(inplace=True)
big_train.drop(columns=["Id"],inplace=True)

In [ ]:
big_train.shape

In [ ]:
def df_add_deltas(df_old):
    df=df_old.copy()
    df=df.sort_values(by=['location', 'Date'])
    df['d_ConfirmedCases'] = df.groupby(['location'])['ConfirmedCases'].diff()
    df['d_Fatalities'] = df.groupby(['location'])['Fatalities'].diff()
    df.loc[df['d_Fatalities']<0,'d_Fatalities']=0
    df.loc[df['d_ConfirmedCases']<0,'d_ConfirmedCases']=0
    
    df['prev_ConfirmedCases']=df['ConfirmedCases']-df['d_ConfirmedCases']
    df['prev_Fatalities']=df['Fatalities']-df['d_Fatalities']
    
    df['prev_ConfirmedCases']=np.log1p(df['prev_ConfirmedCases'])
    df['prev_Fatalities']=np.log1p(df['prev_Fatalities'])
    
    df['prev5_ConfirmedCases'] = df.groupby(['location'])['ConfirmedCases'].shift(5).fillna(0)
    df['prev5_Fatalities'] = df.groupby(['location'])['Fatalities'].shift(5).fillna(0)
    
    df['prev5_ConfirmedCases']=np.log1p(df['prev5_ConfirmedCases'])
    df['prev5_Fatalities']=np.log1p(df['prev5_Fatalities'])
    
    first_day_stat=df[df['Date']=='2020-01-22']
    df.drop(df[df['Date']=='2020-01-22'].index, inplace=True)
    
    return df,first_day_stat
    

In [ ]:
big_train,first_day_stat=df_add_deltas(big_train)

In [ ]:
#big_train.head()

In [ ]:
big_train.reset_index(inplace=True,drop=True)

In [ ]:
X=big_train.drop(columns=['Province_State','Country_Region','Date','ConfirmedCases','Fatalities','location',
                          'd_ConfirmedCases','d_Fatalities'])

y=big_train['d_ConfirmedCases']
y_2=big_train['d_Fatalities']

In [ ]:
max_day=X['day_of_year'].max()
mask_train=X['day_of_year']<max_day-DT+1
mask_test=X['day_of_year']>=max_day-DT+1

In [ ]:
X_train=X[mask_train]
X_test=X[mask_test]


y_train=y[mask_train]
y_test=y[mask_test]

y_train_2=y_2[mask_train]
y_test_2=y_2[mask_test]

In [ ]:
X_test['day_of_year'].nunique()

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

corr = big_train[['d_ConfirmedCases','d_Fatalities','days_passed','ConfirmedCases_neighbors','pop',
                  'medianage','sex65plus','lung','smokers','density','prev_ConfirmedCases','prev_Fatalities']].corr("spearman")


In [ ]:
X_train.drop(columns=['day_of_year'],inplace=True)  #including day of year makes things worse RMLSE goes up from 0.49 to 0.7
X_test.drop(columns=['day_of_year'],inplace=True)   #including day of year makes things worse RMLSE goes up from 0.49 to 0.7

X_train.drop(columns=['day_of_week'],inplace=True)  #including day of week makes things worse RMLSE goes up from 0.49 to 0.57
X_test.drop(columns=['day_of_week'],inplace=True)   #including day of week makes things worse RMLSE goes up from 0.49 to 0.57

X.drop(columns=['day_of_year'],inplace=True)  
X.drop(columns=['day_of_week'],inplace=True)   



In [ ]:
X.drop(columns=['index'],inplace=True)   
X_train.drop(columns=['index'],inplace=True)
X_test.drop(columns=['index'],inplace=True)

In [ ]:
# Best: -0.252369 using {'max_depth': 6, 'n_estimators': 1500}

# Best: -1.051575 using {'max_depth': 6, 'n_estimators': 500} - predict shifts log

# Best: -278.598983 using {'max_depth': 10, 'n_estimators': 500} - predict values

# Best: -1.111758 using {'max_depth': 6, 'n_estimators': 500} - predict shifts log, knowing prev log 

import xgboost as xgb
from sklearn.model_selection import GridSearchCV

if optimize_model:

    model = xgb.XGBRegressor(random_state=42)
    n_estimators_grid = [500, 750,1000]
    max_depth_grid = [4, 6, 8]
    param_grid = dict(max_depth=max_depth_grid, n_estimators=n_estimators_grid)
    grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=[(X[mask_train].index,X[mask_test].index)], verbose=1)
    grid_result = grid_search.fit(X,np.log1p(y))
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    print(grid_result.cv_results_)

In [ ]:
# Best: -0.211438 using {'max_depth': 5, 'n_estimators': 2500}

# Best: -0.974302 using {'max_depth': 5, 'n_estimators': 400} - predict shifts log

# Best: -274.964946 using {'max_depth': 12, 'n_estimators': 500}

# Best: -1.064197 using {'max_depth': 5, 'n_estimators': 400}



if optimize_model:

    model = xgb.XGBRegressor(random_state=42)
    n_estimators_grid = [400,500,600]
    max_depth_grid = [3,4,5]
    param_grid = dict(max_depth=max_depth_grid, n_estimators=n_estimators_grid)
    grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=[(X[mask_train].index,X[mask_test].index)], verbose=1)
    grid_result = grid_search.fit(X,np.log1p(y))
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    print(grid_result.cv_results_)

In [ ]:
#Best: -0.211107 using {'max_depth': 5, 'n_estimators': 3000}

#Best: -0.940498 using {'max_depth': 4, 'n_estimators': 400}

#Best: -274.964946 using {'max_depth': 12, 'n_estimators': 500}

#Best: -0.861262 using {'max_depth': 2, 'n_estimators': 200}

#Best: -0.834393 using {'max_depth': 2, 'n_estimators': 150}


if optimize_model:

    model = xgb.XGBRegressor(random_state=42)
    n_estimators_grid = [125,150,175]
    max_depth_grid = [1,2,3]
    param_grid = dict(max_depth=max_depth_grid, n_estimators=n_estimators_grid)
    grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=[(X[mask_train].index,X[mask_test].index)], verbose=1)
    grid_result = grid_search.fit(X,np.log1p(y))
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    print(grid_result.cv_results_)

In [ ]:
reg = xgb.XGBRegressor(n_estimators=n_estimators,max_depth=max_depth,random_state=42)
reg_2 = xgb.XGBRegressor(n_estimators=n_estimators,max_depth=max_depth,random_state=42)

In [ ]:
reg.fit(X_train,np.log1p(y_train))

In [ ]:
plot = xgb.plot_importance(reg, max_num_features=10)

In [ ]:
y_pred = reg.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_pred,np.log1p(y_test)))

In [ ]:
X_train_2=X_train.copy()
X_train_2['d_confirmed']=y_train  #0.4412899060661785 <- without , with - 0.4463  
X_test_2=X_test.copy()
X_test_2['d_confirmed']=y_pred

In [ ]:
reg_2.fit(X_train_2,np.log1p(y_train_2))

In [ ]:
plot = xgb.plot_importance(reg_2, max_num_features=10)

In [ ]:
y_pred_2 = reg_2.predict(X_test_2)

In [ ]:
np.sqrt(mean_squared_error(y_pred_2,np.log1p(y_test_2)))

In [ ]:
test=pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/test.csv")
test.rename(columns={'ForecastId':'Id'},inplace=True)
test=add_location(test)

test.set_index('location',inplace=True)

test['day_of_year']=test['Date'].dt.dayofyear
test['day_of_week']=test['Date'].dt.dayofweek
test=add_days_passed(test,first_day)
test=add_country_stat(test,country_stat)

In [ ]:
days_to_predict=test['Date'].unique()
days_to_predict.sort()

In [ ]:
test.head()

In [ ]:
big_train=big_train.drop(columns=["index"])


In [ ]:
known=big_train['Date'].unique()

In [ ]:
if Make_submission==True:
    results=[]

    for d in days_to_predict:
        print("Predicting {}".format(d))
        if d in known:
            print("Data Known")
        
            X=big_train.drop(columns=['Province_State','Country_Region','ConfirmedCases','Fatalities','location','Date',
                                  'day_of_year','day_of_week','d_ConfirmedCases','d_Fatalities'])

            y=big_train['d_ConfirmedCases']
            y_2=big_train['d_Fatalities']
        
            mask_train=big_train['Date']<d
            mask_val=big_train['Date']==d
        
            X_train=X[mask_train]
            y_train=y[mask_train]
            y_train_2=y_2[mask_train]
        
            X_val=X[mask_val]
            y_val=y[mask_val]
            y_val_2=y_2[mask_val]
        
            reg = xgb.XGBRegressor(n_estimators=n_estimators,max_depth=max_depth,random_state=42)
            reg_2 = xgb.XGBRegressor(n_estimators=n_estimators,max_depth=max_depth,random_state=42)
        
            reg.fit(X_train,np.log1p(y_train))
        
            y_pred = reg.predict(X_val)
            print("MSLE {}".format(mean_squared_error(y_pred,np.log1p(y_val))))
        
            X_train_2=X_train.copy()
            X_train_2['d_ConfirmedCases']=np.log1p(y_train)  #0.4412899060661785 <- without , with - 0.4463  
            X_val_2=X_val.copy()
            X_val_2['d_ConfirmedCases']=y_pred
        
            reg_2.fit(X_train_2,np.log1p(y_train_2))
        
            y_pred_2 = reg_2.predict(X_val_2)
        
            print("MSLE {}".format(mean_squared_error(y_pred_2,np.log1p(y_val_2))))
        
        #result=X_test[['']]
        elif d-np.timedelta64(86400000000000,'ns') in known:
            print("Data Known")
        
            X=big_train.drop(columns=['Province_State','Country_Region','ConfirmedCases','Fatalities','location','Date',
                                  'day_of_year','day_of_week','d_ConfirmedCases','d_Fatalities'])

            y=big_train['d_ConfirmedCases']
            y_2=big_train['d_Fatalities']
        
            mask_train=big_train['Date']<d
        
            X_train=X[mask_train]
            y_train=y[mask_train]
            y_train_2=y_2[mask_train]
        
        
            reg = xgb.XGBRegressor(n_estimators=n_estimators,max_depth=max_depth,random_state=42)
            reg_2 = xgb.XGBRegressor(n_estimators=n_estimators,max_depth=max_depth,random_state=42)
        
            reg.fit(X_train,np.log1p(y_train))
        
            X_train_2=X_train.copy()
            X_train_2['d_ConfirmedCases']=np.log1p(y_train)  #0.4412899060661785 <- without , with - 0.4463  
            
            reg_2.fit(X_train_2,np.log1p(y_train_2))
        
        
        
        X_test=test[test['Date']==d]
    
        day=X_test['day_of_year'].iloc[0]
    
        country_info=big_train[big_train['day_of_year']==day-1].groupby(['Country_Region'])['ConfirmedCases'].sum()
    
        border_cases=border_info.merge(country_info, how='inner', 
                left_on=['country_border_name'],right_on=['Country_Region'])
    
        border_cases=border_cases.groupby(['country_name'])['ConfirmedCases'].sum()
        border_cases=border_cases.rename('ConfirmedCases_neighbors')
    
        X_test=X_test.merge(border_cases, how='left', left_on=['Country_Region'],right_on=['country_name'])
        X_test['ConfirmedCases_neighbors']=X_test['ConfirmedCases_neighbors'].fillna(0)
    
        X_test = pd.concat([X_test,pd.get_dummies(X_test['location'], prefix='loc')],axis=1)
        X_test['ConfirmedCases_neighbors']=np.log1p(X_test['ConfirmedCases_neighbors'])
        
        X_test=X_test.merge(big_train[big_train['day_of_year']==day-1][['location','ConfirmedCases','Fatalities']], how='left', 
                 left_on=['location'],right_on=['location'])
        X_test.rename(columns={'ConfirmedCases':'prev_ConfirmedCases','Fatalities':'prev_Fatalities'},inplace=True)
        
        X_test['prev_ConfirmedCases']=np.log1p(X_test['prev_ConfirmedCases'])
        X_test['prev_Fatalities']=np.log1p(X_test['prev_Fatalities'])
        
        X_test=X_test.merge(big_train[big_train['day_of_year']==day-5][['location','ConfirmedCases','Fatalities']], how='left', 
                 left_on=['location'],right_on=['location'])
        X_test.rename(columns={'ConfirmedCases':'prev5_ConfirmedCases','Fatalities':'prev5_Fatalities'},inplace=True)
        
        X_test['prev_ConfirmedCases']=np.log1p(X_test['prev_ConfirmedCases'])
        X_test['prev_Fatalities']=np.log1p(X_test['prev_Fatalities'])
        
    
        X_test.set_index('Id',inplace=True)
    
    #print(X_test.head(5))
    
        y_test=reg.predict(X_test.drop(columns=['Province_State','Country_Region','location','Date','day_of_year','day_of_week']))
    
    #print(y_test)
    
        X_test['d_ConfirmedCases']=y_test
    
        y_test=reg_2.predict(X_test.drop(columns=['Province_State','Country_Region','location','Date',
                                            'day_of_year','day_of_week']))
    
        X_test['d_Fatalities']=y_test
    
    #print(X_test.shape)
    
        X_test['Id']=X_test.index
    
        X_test=X_test.merge(big_train[big_train['day_of_year']==day-1][['location','ConfirmedCases','Fatalities']], how='left', 
                 left_on=['location'],right_on=['location'])
    
    #print(X_test.head(5))
    
    #X_test.set_index('Id',inplace=True)
    
    #print(X_test.shape)
    
        X_test.set_index('Id',inplace=True)
    
        print(X_test.head(5))
        
        X_test['d_ConfirmedCases']=np.expm1(X_test['d_ConfirmedCases'])
        X_test['d_Fatalities']=np.expm1(X_test['d_Fatalities'])
    
        X_test['ConfirmedCases']+=X_test['d_ConfirmedCases']
        X_test['Fatalities']+=X_test['d_Fatalities']
       
    
    
        results.append(X_test[['ConfirmedCases','Fatalities']])
    
        if not d in known: #Needed to correctly get data on neighbors         
            big_train=pd.concat([big_train,X_test])
    
    
    
    
    
    
    
    
        
        
        
        

In [ ]:
if Make_submission==True:
    submission=pd.read_csv("../input/covid19-global-forecasting-week-4/submission.csv")
    submission.drop(columns=['ConfirmedCases','Fatalities'],inplace=True)
    submission=submission.merge(pd.concat(results),left_on=['ForecastId'],right_index=True).clip(lower=0)
    submission.to_csv('submission.csv',index=False)